In [1]:
from tkinter import *
from tkinter import messagebox
from PIL import ImageTk,Image
import sqlite3
from tkinter import ttk
from threading import Thread
import numpy as np
import time
import tkinter as tk
import string
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from gensim.test.utils import common_texts
from gensim import downloader as api
from sentence_transformers import SentenceTransformer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
text8_corpus = api.load('text8')
text8_corpus = [doc for doc in text8_corpus]

list_of_list_of_tokens = []
for doc in text8_corpus:
    tokens = [w for w in doc if w not in stop_words]
    list_of_list_of_tokens.append(tokens)

dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
lda_model = LdaModel.load('WebAES_LDA_Model.model') 

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [3]:
def preprocess_text(text):
    text = text.lower()
    
    tokens = word_tokenize(text)
    
    tokens = [word for word in tokens if word not in stop_words]
    
    pos_tagged_tokens = nltk.pos_tag(tokens)
    tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tagged_tokens]
    
    tokens = [stemmer.stem(token) for token in tokens]
    
    return tokens

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN

In [4]:
def topic_distribution_to_vector(topic_dist, num_topics):
    vec = np.zeros(num_topics)
    for topic_id, prob in topic_dist:
        vec[topic_id] = prob
    return vec
def compute_cosine_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

In [5]:
def evaluate(ideal_answer, student_answer,mark):
    print(len(student_answer))
    if len(student_answer)>1:
        student_answer_processed = preprocess_text(student_answer)
        ideal_answer_processed = preprocess_text(ideal_answer)

        student_bow = dictionary_LDA.doc2bow(student_answer_processed)
        ideal_bow = dictionary_LDA.doc2bow(ideal_answer_processed)



        student_topic_dist = lda_model.get_document_topics(student_bow, minimum_probability=0)
        ideal_topic_dist = lda_model.get_document_topics(ideal_bow, minimum_probability=0)

        num_topics = lda_model.num_topics

        student_topic_vector = topic_distribution_to_vector(student_topic_dist, num_topics)
        ideal_topic_vector = topic_distribution_to_vector(ideal_topic_dist, num_topics)

        cos_sim_lda = compute_cosine_similarity(student_topic_vector, ideal_topic_vector)

        ideal_embedding = sbert_model.encode([ideal_answer])
        student_embedding = sbert_model.encode([student_answer])

        cos_sim_sbert = cosine_similarity(student_embedding, ideal_embedding)[0][0]

        total_mark=cos_sim_lda*0.5*mark+cos_sim_sbert*0.5*mark
    else:
        total_mark=0
    
    return total_mark


In [6]:
ans=0
text=''
file=""
fileQ=""
Qtext=""
strans=""
ansl=[]

In [7]:
def openmyfile(x):
    global count
    global no
    global fileQ
    global Qtext
    global ansl
    global X
    global m
    X=x
    if x=='AI':
        y='artificial.db'
    elif x=='OS':
        y='os.db'
    elif x=='DATA STRUCTURE':
        y='data_structure.db'
    elif x=='CLOUD':
        y='cloud.db'
    elif x=='PYTHON':
        y='python.db'
        
    with sqlite3.connect(y) as db:
        c = db.cursor()
        find_sub = """SELECT * from ques"""
        c.execute(find_sub)
        result = c.fetchall()
    no=len(result)
    #print(no)
    Qtext=[]
    ansl=[]
    if count<no:
        Qtext.append(result[count][1])
        ansl.append(result[count][2])
        m=result[count][3]

In [8]:
with sqlite3.connect('log_info.db') as db:
    c = db.cursor()

c.execute('CREATE TABLE IF NOT EXISTS user (username TEXT NOT NULL PRIMARY KEY,password TEX NOT NULL);')
db.commit()
db.close()

In [9]:
with sqlite3.connect('teacher_info.db') as db:
    c = db.cursor()

c.execute('CREATE TABLE IF NOT EXISTS teacher (username TEXT NOT NULL PRIMARY KEY,password TEX NOT NULL);')
db.commit()
db.close()

In [10]:
with sqlite3.connect('exam_system.db') as db:
    c = db.cursor()

c.execute('CREATE TABLE IF NOT EXISTS exam_requests (id INTEGER AUTO INCREMENT,username TEXT NOT NULL PRIMARY KEY,status TEXT NOT NULL);')
db.commit()
db.close()

In [11]:
class Login_System:
    def __init__(self):
        self.root=Tk()
        self.root.title("Home")
        self.root.geometry("1200x700")
        self.root.iconbitmap("icon.ico")

        self.bg_icon=ImageTk.PhotoImage(file="img2.jpg")
        #self.user_icon=PhotoImage(file="user.png")
        self.user1_icon=PhotoImage(file="user.png")
        self.pass_icon=PhotoImage(file="pass.png")
        self.logo_icon=PhotoImage(file="logo.png")
        self.logbttn_icon=PhotoImage(file="log_bttn.png")
        
        self.sigbttn_icon=PhotoImage(file="sig.png")
        
        self.uname=StringVar()
        self.pass_=StringVar()
        self.nuname=StringVar()
        self.npass_=StringVar()


        bg_lbl=Label(self.root,image=self.bg_icon).pack()
        
        title=Label(self.root,text='DESCRIPTIVE ANSWER CHECKING SYSTEM',font=('times new roman',40,'bold'),bg='white',fg='blue2',bd=10,relief=SOLID)
        title.place(x=0,y=0,relwidth=1)

        
        self.Login_Frame=Frame(self.root,bg='white')
        self.Login_Frame.place(x=350,y=150)
        
        self.role_var = tk.StringVar()
        self.role_var.set("Student")  
        self.selected_role='Student'
        #print(self.selected_role)
        # Create a Combobox for role selection
        role=Label(self.Login_Frame,text='Role',font=('times new roman',20,'bold'),bg='white').grid(row=1,column=0,padx=20,pady=10)
        self.role_combobox = ttk.Combobox(self.Login_Frame, textvariable=self.role_var, font=("times new roman", 15))
        self.role_combobox["values"] = ("Student", "Teacher","Admin")  # Set available roles
        self.role_combobox.grid(row=1, column=1, padx=20,pady=10)
        self.role_combobox.bind("<<ComboboxSelected>>", self.on_role_selected)
        
        logolbl=Label(self.Login_Frame,image=self.logo_icon,bd=0).grid(row=0,columnspan=2,pady=20)
        lbluser=Label(self.Login_Frame,text='Username',image=self.user1_icon,compound=LEFT,font=('times new roman',20,'bold'),bg='white').grid(row=2,column=0,padx=20,pady=10)
        txtuser=Entry(self.Login_Frame,bd=1,textvariable=self.uname,relief=SOLID,font=("",15)).grid(row=2,column=1,padx=20)
        lblpass=Label(self.Login_Frame,text='Password',image=self.pass_icon,compound=LEFT,font=('times new roman',20,'bold'),bg='white').grid(row=3,column=0,padx=20,pady=10)
        txtpass=Entry(self.Login_Frame,bd=1,textvariable=self.pass_,relief=SOLID,font=("",15),show='*').grid(row=3,column=1,padx=20)
        Button(self.Login_Frame,text="Don't have an Account? Create One",command=self.userlogin,font=('times new roman',14,'bold'),bg='white',fg='black',borderwidth=0).grid(row=4,columnspan=2,pady=10)
        button = Button(self.Login_Frame,borderwidth=0,command=self.login)
        button.config(image=self.logbttn_icon,bg='white')
        button.grid(row=5,columnspan=2,pady=10)

        self.user_Frame=Frame(self.root,bg='white')
        
        role=Label(self.user_Frame,text='Role',font=('times new roman',20,'bold'),bg='white').grid(row=1,column=0,padx=20,pady=10)
        self.role_combobox = ttk.Combobox(self.user_Frame, textvariable=self.role_var, font=("times new roman", 15))
        self.role_combobox["values"] = ("Student", "Teacher","Admin")  # Set available roles
        self.role_combobox.grid(row=1, column=1, padx=20,pady=10)
        self.role_combobox.bind("<<ComboboxSelected>>", self.on_role_selected)
        
        lbluser=Label(self.user_Frame,text='Username',image=self.user1_icon,compound=LEFT,font=('times new roman',20,'bold'),bg='white').grid(row=2,column=0,padx=20,pady=10)
        txtuser=Entry(self.user_Frame,bd=1,textvariable=self.nuname,relief=SOLID,font=("",15)).grid(row=2,column=1,padx=20)
        lblpass=Label(self.user_Frame,text='Password',image=self.pass_icon,compound=LEFT,font=('times new roman',20,'bold'),bg='white').grid(row=3,column=0,padx=20,pady=10)
        txtpass=Entry(self.user_Frame,bd=1,textvariable=self.npass_,relief=SOLID,font=("",15),show='*').grid(row=3,column=1,padx=20)
        Button(self.user_Frame,text="Already have an Account? Log In",command=self.log,font=('times new roman',14,'bold'),bg='white',fg='black',borderwidth=0).grid(row=4,columnspan=2,pady=10)
        button = Button(self.user_Frame,borderwidth=0,command=self.new_user)
        button.config(image=self.sigbttn_icon,bg='white')
        button.grid(row=5,columnspan=2,pady=10)
        self.root.mainloop()

    def on_role_selected(self, event):
        # This function will be called when the user selects an option from the combobox
        self.selected_role = self.role_var.get()
        #print("Selected role:", self.selected_role)  # You can update or use the role_var here as needed
    
         
    def log(self):
        self.uname.set('')
        self.pass_.set('')
        self.user_Frame.place_forget()
        self.Login_Frame.place(x=350,y=150)
        
    def userlogin(self):
        self.nuname.set('')
        self.npass_.set('')
        self.Login_Frame.place_forget()
        self.user_Frame.place(x=350,y=150)
        
        self.user_icon=PhotoImage(file="user_log.png")
        logolbl=Label(self.user_Frame,image=self.user_icon,bd=0,bg='white').grid(row=0,columnspan=2,pady=20)
        

    def new_user(self):
        if self.selected_role=='Student':
            with sqlite3.connect('log_info.db') as db:
                c = db.cursor()
            find_user = ('SELECT username FROM user WHERE username = ?')
            c.execute(find_user,[self.nuname.get()])
        elif self.selected_role=='Teacher':
            with sqlite3.connect('teacher_info.db') as db:
                c = db.cursor()
            find_user = ('SELECT username FROM teacher WHERE username = ?')
            c.execute(find_user,[self.nuname.get()])
        
        

        if(self.nuname.get()== '' or self.npass_.get()==''):
            messagebox.showerror('Oops!','Please complete the required field!.')
            
        elif c.fetchall():
            messagebox.showerror('Error!','Username Taken Try a Diffrent One.')
        
        else:
            password = self.npass_.get()

            # Password must be at least 8 characters and contain at least one special character
            if len(password) < 8:
                messagebox.showerror("Error", "Password must be at least 8 characters long!")
                return
            if not re.search(r"[!@#$%^&*(),.?\":{}|<>]", password):
                messagebox.showerror("Error", "Password must contain at least one special character!")
                return
            messagebox.showinfo('Success!','Account Created!')
            if self.selected_role=='Student':
                insert = 'INSERT INTO user(username,password) VALUES(?,?)'
                c.execute(insert,[self.nuname.get(),self.npass_.get()])
                db.commit()
            if self.selected_role=='Teacher':
                insert = 'INSERT INTO teacher(username,password) VALUES(?,?)'
                c.execute(insert,[self.nuname.get(),self.npass_.get()])
                db.commit()
            self.log()
        
            
       
    def login(self):
        self.a=0
        
        if self.selected_role=='Admin':
            self.root.destroy()
            if(self.uname.get()== 'admin' or self.pass_.get()=='admin'):
                
                AdminDashboard()
       
        elif self.selected_role=='Student':
            with sqlite3.connect('log_info.db') as db:
                c = db.cursor()
            find_user = ('SELECT * FROM user WHERE username = ? and password = ?')
            c.execute(find_user,[self.uname.get(),self.pass_.get()])
            result = c.fetchall()
        elif self.selected_role=='Teacher':
            with sqlite3.connect('teacher_info.db') as db:
                c = db.cursor()
            find_user = ('SELECT * FROM teacher WHERE username = ? and password = ?')
            c.execute(find_user,[self.uname.get(),self.pass_.get()])
            result = c.fetchall()
            

        
        if(self.uname.get()== '' or self.pass_.get()==''):
        
            messagebox.showerror('Oops!','Please complete the required field!.')
        elif(result):
           
            self.a=1
            if self.uname.get()=='admin':
                markview(self.uname.get())
            else:
                self.root.destroy()
                if self.selected_role=='Student':
                    settings(self.uname.get())
                if self.selected_role=='Teacher':
                    markview(self.uname.get())
                
            
            
            
        elif(self.uname.get()== '' or self.pass_.get()==''):
        
            messagebox.showerror('Oops!','Please complete the required field!.')
            
        elif(self.a==0):
            messagebox.showerror('Oops!','Username Not Found.')
            self.uname.set('')
            self.pass_.set('')

In [12]:
class AdminDashboard(tk.Tk):
    def __init__(self):
        super().__init__()
        title=Label(self,text='DESCRIPTIVE ANSWER CHECKING SYSTEM',font=('times new roman',40,'bold'),bg='white',fg='blue2',bd=10,relief=SOLID)
        title.place(x=0,y=0,relwidth=1)
        self.geometry('1200x700')
        self.configure(background='light gray')  # Background color for the main window

        # Set the style of the widgets
        style = ttk.Style()
        style.configure('TFrame', background='light gray')
        style.configure('TButton', background='light blue', font=('Arial', 10))
        style.configure('TLabel', background='light gray', font=('Arial', 12))

        # Refresh Button
        self.refresh_button = ttk.Button(self, text="Refresh", command=self.load_pending_requests)
        self.refresh_button.pack(pady=95)
        
        self.logout_button = ttk.Button(self, text="Logout", command=self.logout)
        self.logout_button.pack(pady=2)

        # Frame for requests
        self.requests_frame = ttk.Frame(self, style='TFrame')
        self.requests_frame.pack(fill='both', expand=True)

        self.load_pending_requests()
        
        self.requests.mainloop()
    def load_pending_requests(self):
        # Clear existing widgets
        for widget in self.requests_frame.winfo_children():
            widget.destroy()

        # Database operation
        conn = sqlite3.connect('exam_system.db')
        cursor = conn.cursor()
        cursor.execute("SELECT id, username FROM exam_requests WHERE status = 'requested'")
        pending_requests = cursor.fetchall()
        conn.close()

        # Display pending requests
        for idx, (request_id, username,) in enumerate(pending_requests, start=1):
            row_frame = ttk.Frame(self.requests_frame, style='TFrame')
            row_frame.pack(fill='x', padx=50, pady=15)

            ttk.Label(row_frame, text=f"{idx}. Student Name: {username}").pack(side='left')

            approve_button = ttk.Button(row_frame, text="Approve", command=lambda req_id=request_id: self.approve_request(username))
            approve_button.pack(side='right')

    def approve_request(self, username):
        approve_exam_request(username)
        self.load_pending_requests()
        
    def logout(self):
        # This method will close the application
        self.destroy()
        Login_System()

def approve_exam_request(username):
    conn = sqlite3.connect('exam_system.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE exam_requests SET status = 'approved' WHERE username = ?", (username,))
    conn.commit()
    conn.close()


In [13]:
class markview():
    def __init__(self,username):
        global count
        
        self.name=username
        
        self.set=tk.Tk()
        self.set.title("Mark Viewer")
        self.set.geometry("1200x700")
        self.set.configure(bg='lightgreen')
        self.set.iconbitmap("set.ico")
        title=Label(self.set,text='WELCOME '+username,font=('times new roman',40,'bold'),bg='white',fg='blue2',bd=10,relief=SOLID)
        title.place(x=0,y=0,relwidth=1)
        self.sub_var = tk.StringVar()
        self.sub_var.set("AI")  
        self.selected_sub='AI'
        #print(self.selected_role)
        # Create a Combobox for role selection
        self.flag=0
        role=Label(self.set,text='Select Subject',font=('times new roman',20,'bold'),bg='lightgreen').place(x=50,y=100)
        self.sub_combobox = ttk.Combobox(self.set, textvariable=self.sub_var, font=("times new roman", 15))
        self.sub_combobox["values"] = ('AI','OS','DATA STRUCTURE','CLOUD','PYTHON')  
        self.sub_combobox.place(x=300,y=110)
        self.sub_combobox.bind("<<ComboboxSelected>>", self.on_sub_selected)
        
        self.update_button = tk.Button(self.set, text="Update",bg='grey', font=("times new roman", 14,'bold'), command=self.update_ques)
        self.update_button.place(x=600,y=100)

        self.view_report_button = tk.Button(self.set, text="View Report",bg='grey', font=("times new roman", 14,'bold'), command=self.view_mark)
        self.view_report_button.place(x=750,y=100)

        self.logout_button = tk.Button(self.set, text="Logout",bg='grey', font=("times new roman", 14,'bold'), command=self.logout)
        self.logout_button.place(x=900,y=100)
        
        self.question_index = 0
        

        # Create Text widgets for question and answer
        self.question_textbox = tk.Text(self.set, width=50, height=5)
        

        self.answer_textbox = tk.Text(self.set, width=75, height=10)
        


        # Create an entry widget for the mark
        self.mark_entry = tk.Entry(self.set, width=10)
        
        
        

        # Create buttons to navigate and update fields
        self.previous_button = tk.Button(self.set, text="Previous", command=self.on_previous)
        

        self.submit_button = tk.Button(self.set, text="Submit", command=self.on_submit)
        

        self.next_button = tk.Button(self.set, text="Next", command=self.on_next)
        

        # Display the first question on application start
        
        
        self.set.mainloop()
    def on_sub_selected(self, event):
        # This function will be called when the user selects an option from the combobox
        self.selected_sub = self.sub_var.get()
        print("Selected sub:", self.selected_sub)  # You can update or use the role_var here as needed
        self.questions = self.fetch_question()
        self.display_question()
        
        
    def view_mark(self):
        if self.flag==1:
            
            self.question_textbox.place_forget()
            self.answer_textbox.place_forget()
            self.mark_entry.place_forget()
            self.previous_button.place_forget()
            self.submit_button.place_forget()
            self.next_button.place_forget()
        self.flag=1
        conn = sqlite3.connect(f"{self.selected_sub}.db")
        cursor = conn.cursor()

        # Execute the SELECT query
        cursor.execute("SELECT * FROM mark")
        rows = cursor.fetchall()

        # Create a Treeview widget to display the data
        self.tree = ttk.Treeview(self.set)
        self.tree["columns"] = tuple(range(len(rows[0])))

        # Add columns to the Treeview
        for i, column_name in enumerate(cursor.description):
            column_name = column_name[0]
            self.tree.column(i, width=75, anchor="center")
            self.tree.heading(i, text=column_name)

        # Add data rows to the Treeview
        for row in rows:
            self.tree.insert("", "end", values=row)

        # Pack the Treeview widget
        self.tree.place(x=20,y=250)

        # Close the cursor and the database connection
        cursor.close()
        conn.close()

    def update_ques(self):
        # Function to view student report
        # Implement your logic here
        if self.flag==1:
            self.tree.place_forget()
        self.flag=1
        
        self.question_textbox.place(x=400,y=250)
        self.answer_textbox.place(x=300,y=400)
        self.mark_entry.place(x=600,y=600)
        self.previous_button.place(x=500,y=650)
        self.submit_button.place(x=600,y=650)
        self.next_button.place(x=700,y=650)
        

        print("Viewing report...")

    def logout(self):
        # Function to handle logout
        # Implement your logic here
        print("Logging out...")
        self.set.destroy()  
     
    
    def fetch_question(self):
        print(self.sub_var.get())
        if self.sub_var.get()=='AI':
            self.sub='artificial.db'
        elif self.sub_var.get()=='OS':
            self.sub='os.db'
        elif self.sub_var.get()=='DATA STRUCTURE':
            self.sub='data_structure.db'
        elif self.sub_var.get()=='CLOUD':
            self.sub='cloud.db'
        elif self.sub_var.get()=='PYTHON':
            self.sub='python.db'
        with sqlite3.connect(self.sub) as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM ques')
            data = cursor.fetchall()
            return data

    def update_question(self,selected_question_id, new_question):
        if self.selected_sub=='AI':
            self.sub='artificial.db'
        elif self.selected_sub=='OS':
            self.sub='os.db'
        elif self.selected_sub=='DATA STRUCTURE':
            self.sub='data_structure.db'
        elif self.selected_sub=='CLOUD':
            self.sub='cloud.db'
        elif self.selected_sub=='PYTHON':
            self.sub='python.db'
        with sqlite3.connect(self.sub) as conn:
            cursor = conn.cursor()
            cursor.execute('UPDATE ques SET QUESTION=? WHERE NO=?', (new_question, selected_question_id))
            conn.commit()

    def update_answer(self,selected_question_id, new_answer):
        if self.selected_sub=='AI':
            self.sub='artificial.db'
        elif self.selected_sub=='OS':
            self.sub='os.db'
        elif self.selected_sub=='DATA STRUCTURE':
            self.sub='data_structure.db'
        elif self.selected_sub=='CLOUD':
            self.sub='cloud.db'
        elif self.selected_sub=='PYTHON':
            self.sub='python.db'
        with sqlite3.connect(self.sub) as conn:
            cursor = conn.cursor()
            cursor.execute('UPDATE ques SET ANSWER=? WHERE NO=?', (new_answer, selected_question_id))
            conn.commit()

    def update_mark(self,selected_question_id, new_mark):
        if self.selected_sub=='AI':
            self.sub='artificial.db'
        elif self.selected_sub=='OS':
            self.sub='os.db'
        elif self.selected_sub=='DATA STRUCTURE':
            self.sub='data_structure.db'
        elif self.selected_sub=='CLOUD':
            self.sub='cloud.db'
        elif self.selected_sub=='PYTHON':
            self.sub='python.db'
        with sqlite3.connect(self.sub) as conn:
            cursor = conn.cursor()
            cursor.execute('UPDATE ques SET MARK=? WHERE NO=?', (new_mark, selected_question_id))
            conn.commit()

    def display_question(self):
        selected_question_id, question_text, answer, mark = self.questions[self.question_index]
        self.question_textbox.delete('1.0', tk.END)
        self.question_textbox.insert('1.0', question_text)

        self.answer_textbox.delete('1.0', tk.END)
        self.answer_textbox.insert('1.0', answer)

        self.mark_entry.delete(0, tk.END)
        self.mark_entry.insert(0, mark)

    def on_submit(self):
        new_question = self.question_textbox.get('1.0', tk.END).strip()
        new_answer = self.answer_textbox.get('1.0', tk.END).strip()
        new_mark = int(self.mark_entry.get())

        selected_question_id = self.questions[self.question_index][0]
        self.update_question(selected_question_id, new_question)
        self.update_answer(selected_question_id, new_answer)
        self.update_mark(selected_question_id, new_mark)

        self.display_question()

    def on_previous(self):
        #global self.question_index
        self.question_index -= 1
        if self.question_index < 0:
            self.question_index = len(self.questions) - 1
        self.display_question()

    def on_next(self):
        #global self.question_index
        self.question_index += 1
        if self.question_index >= len(self.questions):
            self.question_index = 0
        self.display_question()


In [32]:
class settings():
    def __init__(self,username):
        global count
        global no
        self.name=username
        
        self.set=Tk()
        self.set.title("Settings")
        self.set.geometry("1200x700")
        self.set.iconbitmap("set.ico")
        self.bg_icon=ImageTk.PhotoImage(file="img3.jpg")
        self.start_icon=PhotoImage(file="start.png")
        self.save_icon=PhotoImage(file="save.png")
        self.logo_icon=PhotoImage(file="user1.png")
        self.pass_icon=PhotoImage(file="pass.png")
        bg_lbl=Label(self.set,image=self.bg_icon).grid()
        title=Label(self.set,text='Dashboard ',font=('times new roman',40,'bold'),bg='white',fg='blue2',bd=10,relief=SOLID)
        title.place(x=0,y=0,relwidth=1)
        self.set_Frame=Frame(self.set,bg='white',width=800, height=600)
        self.set_Frame.place(x=150,y=100)
        
        self.left_frame = Frame(self.set_Frame, bg='#e4c1f9', width=300, height=600)
        self.left_frame.grid(row=0, column=0, sticky="nsew")

        self.right_frame = Frame(self.set_Frame, bg='white', width=500, height=600)
        self.right_frame.grid(row=0, column=1, sticky="nsew")
        
        self.right_frame_mark = Frame(self.set_Frame, bg='white', width=800, height=550)
        self.right_frame_uppass = Frame(self.set_Frame, bg='white', width=800, height=550)
        
        user_lbl=Label(self.left_frame,image=self.logo_icon,bg='#e4c1f9').grid(pady=20)
        self.user_label = Label(self.left_frame, text="Name: "+username,fg='blue2',bg='#e4c1f9',font=("Arial", 18))
        self.user_label.grid(pady=30)
        self.home_button = Button(self.left_frame, text="HOME", bg='#a9def9', font=("Arial", 14), command=self.home)
        self.home_button.grid(pady=20)
        self.req_button = Button(self.left_frame, text="EXAM REQUEST", bg='#a9def9', font=("Arial", 14), command=self.req)
        self.req_button.grid(padx=10,pady=5)
        req_label=Label(self.left_frame, text="please wait for admin approve the request...",font=('times new roman',10),fg='red',bg='#e4c1f9')
        req_label.grid(padx=10,pady=20)
        self.mark_button = Button(self.left_frame, text="View Mark", bg='#a9def9', font=("Arial", 14),command=self.st_view_mark)
        self.mark_button.grid(padx=10,pady=10)
        self.mark_button = Button(self.left_frame, text="Change Password", bg='#a9def9', font=("Arial", 14),command=self.update_pass)
        self.mark_button.grid(padx=10,pady=10)
        
        self.name_label = Label(self.right_frame, text="Welcome",fg='blue2',bg='white',font=("Arial", 18))
        self.name_label.grid(row = 3, column =1,padx=300)
        count=0
        tkt = StringVar()
        choices = {'AI','OS','DATA STRUCTURE','CLOUD','PYTHON'}
        tkt.set('AI')
        self.file='AI'
        openmyfile(self.file)
        
        popupMenu = OptionMenu(self.right_frame, tkt, *choices)
        Label(self.right_frame, text="Subject     : ",font=('times new roman',20,'bold'),bg='white').place(x=200,y=80)
        popupMenu.place(x=400,y=80)
        
        
        button = Button(self.right_frame,borderwidth=0,command=self.start)
        button.config(image=self.start_icon,bg='white',state=DISABLED)
        #button.grid(row=5,columnspan=2,pady=10)
        button.grid(row=5,columnspan=2,pady=200)
        self.logout_button = Button(self.left_frame, text="Logout",bg='#a9def9', font=("Arial", 14), command=self.logout)
        self.logout_button.grid(pady=10)
        with sqlite3.connect('exam_system.db') as db:
            c = db.cursor()
        find_req = """SELECT * from exam_requests WHERE username = ?"""
        c.execute(find_req,[self.name])
        result = c.fetchall()
        if result:
            data=result[0][2]
            print(data)
            if data=='approved':
                req_label.config(text='Request Approved',fg='green',bg='#e4c1f9')
                button.config(image=self.start_icon,bg='white',state=ACTIVE)
                Label(self.right_frame, text="START",font=('times new roman',15,'bold'),bg='white').place(x=300,y=350)
        self.new_Frame=Frame(self.set,bg='white')
        global file
        global loc
        global Qtext
        #global sheet
        #global wb
        global frf
        global ktf
        global cmf
        global gmf
        global m
        self.hours = int(0)
        self.minutes = int(1)
        #self.file=tkt.get()
        #print(self.file)
        def create_sub(total_questions):
            with sqlite3.connect(f'{self.file}.db') as db:
                c = db.cursor()

            columns = "username TEXT NOT NULL, "
            columns += ", ".join([f"A{i+1} TEXT NULL" for i in range(total_questions)]) + ", "
            columns += ", ".join([f"Q{i+1} INTEGER NULL" for i in range(total_questions)]) + ", "
            columns += "Total INTEGER NULL"

            c.execute(f'CREATE TABLE IF NOT EXISTS mark ({columns});')
            db.commit()
            db.close()
            
            with sqlite3.connect(f'{self.file}.db') as db:
                c = db.cursor()
            insert = 'INSERT INTO mark(username) VALUES(?)'
            c.execute(insert,[self.name])
            db.commit()
        create_sub(no)
        def change_dropdown(*args):
            self.file=tkt.get()
            print(self.file)
            openmyfile(self.file)
            create_sub(no)
        tkt.trace('w', change_dropdown)
        
        
        
        
        #db.close()
        self.set.mainloop()
    def logout(self):
        # Function to handle logout
        # Implement your logic here
        print("Logging out...")
        self.set.destroy()
        Login_System()
        
    
        
    def start(self):
        global count
        self.set_Frame.place_forget()
        self.new_Frame.place(x=200,y=200)
        
        self.timer_label = Label(self.set, text="Time Left: 00:00:00", font=('Arial', 14), bg='white')
        self.timer_label.place(x=1000, y=100)
        if count==0:
            try:
                self.total_seconds = self.hours * 3600 + self.minutes * 60
                self.update_timer()
            except ValueError:
                messagebox.showerror("Invalid Input", "Please enter valid numbers for hours and minutes.")
        if self.total_seconds ==0:
            count=no
            self.start()
        global Qtext
        #print(Qtext[0])
        if count==no-1:
            self.button.grid_forget()
            self.label=tk.Label(self.new_Frame, text=Qtext[0])
            self.label.grid(row= 0, column=0, padx=13, pady=21,sticky=W)
            self.label1=tk.Label(self.new_Frame, text='(Mark = '+str(m)+')')
            self.label1.place(x=50,y=150)
            self.entryA = Text(self.new_Frame, height=14,width=50, padx=5, pady=5, wrap=WORD, background='white')
            self.entryA.grid(row= 0, column=1, padx=13, pady=21,sticky=E)
            self.button1=tk.Button(self.new_Frame,text='  Submit  ',default='active',command=self.click_ok)
            self.button1.grid(row= 1, column=1, padx=13, pady=21)
        if count==no:
            with sqlite3.connect(f'{self.file}.db') as db:
                c = db.cursor()

                find_mark = "SELECT * from mark WHERE username = ?"
                c.execute(find_mark, [self.name])
                result = c.fetchone()

                if result:
                    total = sum(float(result[i]) for i in range(no+1, no+1 + no) if result[i] is not None)

                    sql_update_query = "UPDATE mark SET Total = ? WHERE username = ?"
                    column_values = (total, self.name)
                    c.execute(sql_update_query, column_values)
                    db.commit()
            #db.close()
            self.set.destroy()
            settings(self.name)
            
        if count<no-1:
            self.label=tk.Label(self.new_Frame, text=Qtext[0])
            self.label.grid(row= 0, column=0, padx=13, pady=21,sticky=W)
            self.label1=tk.Label(self.new_Frame, text='(Mark = '+str(m)+')')
            self.label1.place(x=50,y=150)
            self.entryA = Text(self.new_Frame, height=14,width=50, padx=5, pady=5, wrap=WORD, background='white')
            self.entryA.grid(row= 0, column=1, padx=13, pady=21,sticky=E)
            #Bframe= tk.Frame(self)                  
            #Bframe.pack(padx=20,pady=20, anchor='e')
            self.button=tk.Button(self.new_Frame,text='  Next  ',default='active',command=self.click_ok)
            self.button.grid(row= 1, column=1, padx=13, pady=21)
        
    def update_timer(self):
        """Updates the timer label every second."""
        if self.total_seconds > 0:
            hrs, remainder = divmod(self.total_seconds, 3600)
            mins, secs = divmod(remainder, 60)
            self.timer_label.config(text=f"Time Left: {hrs:02}:{mins:02}:{secs:02}")
            self.total_seconds -= 1
            self.set.after(1000, self.update_timer)
        else:
            self.timer_label.config(text="Time's Up!")
            messagebox.showinfo("Exam Over", "Time is up!")
            with sqlite3.connect(f'{self.file}.db') as db:
                c = db.cursor()

                find_mark = "SELECT * from mark WHERE username = ?"
                c.execute(find_mark, [self.name])
                result = c.fetchone()

                if result:
                    total = sum(float(result[i]) for i in range(no+1, no+1 + no) if result[i] is not None)

                    sql_update_query = "UPDATE mark SET Total = ? WHERE username = ?"
                    column_values = (total, self.name)
                    c.execute(sql_update_query, column_values)
                    db.commit()
            #db.close()
            self.set.destroy()
            settings(self.name)
           
    
    def click_ok(self):
        #print("Working Successfully")
        global count
        count=count+1
        global text
        global ans_text
        text= self.entryA.get("1.0",END)
        ans_text=text
        self.label.grid_forget()
        self.newWindow = Report(self.name,self.file)
        print(self.file)
        openmyfile(self.file)
        self.start()
            #self.root.withdraw()
            #self.hide()
        #self.withdraw()
        
        
    def req(self):
        with sqlite3.connect('exam_system.db') as db:
            c = db.cursor()
        find_user = ('SELECT username FROM exam_requests WHERE username = ?')
        c.execute(find_user,[self.name])
        if c.fetchall():
            messagebox.showerror('warning!','Request allready send')
        
        else:
            messagebox.showinfo('Success!','Request send successfully')
            insert = 'INSERT INTO exam_requests(username,status) VALUES(?,?)'
            c.execute(insert,[self.name,'requested'])
            db.commit()
            
    def st_view_mark(self):
        self.right_frame.grid_forget()
        self.right_frame_uppass.grid_forget()
        self.right_frame_mark.grid(row=0, column=1, sticky="nsew")
        self.st_sub_var = tk.StringVar()
        self.st_sub_var.set("AI")  
        self.selected_sub='AI'
        Label(self.right_frame_mark,text='Select Subject:',font=('times new roman',20,'bold'),bg='white').place(x=75,y=105)
        self.st_sub_combobox = ttk.Combobox(self.right_frame_mark, width =15,textvariable=self.st_sub_var, font=("times new roman", 15))
        self.st_sub_combobox["values"] = ('AI','OS','DATA STRUCTURE','CLOUD','PYTHON')  
        self.st_sub_combobox.place(x=300,y=110)
        self.st_sub_combobox.bind("<<ComboboxSelected>>", self.on_st_sub_selected)
        self.sub_button = Button(self.right_frame_mark, text="Submit", bg='#f38375', font=("Arial", 14),command=self.fetch_mark)
        self.sub_button.place(x=550,y=100)
    
    def update_pass(self):
        self.right_frame.grid_forget()
        self.right_frame_mark.grid_forget()
        self.uppass_=StringVar()
        self.right_frame_uppass.grid(row=0, column=1, sticky="nsew")
        uppass=Label(self.right_frame_uppass,text='New Password',image=self.pass_icon,compound=LEFT,font=('times new roman',20,'bold'),bg='white').grid(row=3,column=0,padx=20,pady=10)
        txtpass=Entry(self.right_frame_uppass,bd=1,textvariable=self.uppass_,relief=SOLID,font=("",15),show='*').grid(row=3,column=1,padx=20)
        self.pass_button = Button(self.right_frame_uppass, text="Submit", bg='#f38375', font=("Arial", 14),command=self.new_pass)
        self.pass_button.grid(row=4,column=1,padx=20)
        
    def on_st_sub_selected(self, event):
        # This function will be called when the user selects an option from the combobox
        self.st_selected_sub = self.st_sub_var.get()
        print("Selected sub:", self.st_selected_sub)  # You can update or use the role_var here as needed
        
    def fetch_mark(self):
        conn = sqlite3.connect(f"{self.st_selected_sub}.db")
        cursor = conn.cursor()
        cursor.execute("SELECT Total FROM mark WHERE username = ?",[self.name])
        result = cursor.fetchone()
        if result:
            print(result[0])
            Label(self.right_frame_mark,text=f'Subject: {self.st_selected_sub}',font=('times new roman',20,'bold'),bg='white').place(x=100,y=200)
            Label(self.right_frame_mark,text=f'Total mark: {round(result[0],2)}',font=('times new roman',20,'bold'),bg='white').place(x=100,y=250)
    
    def new_pass(self):
        
        if(self.uppass_.get())== '' :
            messagebox.showerror('Oops!','Please complete the required field!.')
            
        else:
            new_password=self.uppass_.get()
           
            if len(new_password) < 8:
                messagebox.showerror("Error", "Password must be at least 8 characters long!")
                return
            if not re.search(r"[!@#$%^&*(),.?\":{}|<>]", new_password):
                messagebox.showerror("Error", "Password must contain at least one special character!")
                return
            with sqlite3.connect('log_info.db') as db:
                c = db.cursor()
            sql_update_query = "UPDATE user SET password = ? WHERE username = ?"
            column_values = (new_password, self.name)
            c.execute(sql_update_query, column_values)
            db.commit()
            messagebox.showinfo('Success!','Password Updated')
           
    
    def home(self):
        self.right_frame_mark.grid_forget()
        self.right_frame_uppass.grid_forget()
        self.right_frame.grid(row=0, column=1, sticky="nsew")
        
        

In [33]:
class Report(tk.Frame):

    def __init__(self,username,subject):
        global eve
        self.name=username
        self.sub=subject
        global ans
        ans=0
        global ansl
        global text
        #global ev
        print(f'ideal answer:{ansl[0]}')
        print(f'student answer:{text}')
        
        global gm
        global g
        global ma
        global m
        g=0
        answer=text
        ma=evaluate(ansl[0], text,m)
        print(f'mark:{ma}')
        ansl.clear()
        ans=0
        global count
        global x
        print(count)
        
        global ans_text
        with sqlite3.connect(f'{subject}.db') as db:
            c = db.cursor()
        if(count>0 and count<no+1):
            self.sql_update_query = f"""Update mark set Q{count} = ?, A{count}= ?  where username = ?"""
            self.columnValues = (ma,ans_text,self.name)
            c.execute(self.sql_update_query,self.columnValues)
            db.commit()
            db.close()
        
       

In [34]:
Login_System()

approved
